In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(magrittr)
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"
IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/input/"

X <- readRDS(paste(OUT_DIR, "rxn_pca_nls0.03vs0_time_course.Rds", sep = ""))
Y <- readRDS(paste(OUT_DIR,"tcdd_dose_detail_vec_train0.03vs0_time_course.Rds",sep=""))
E <- read.table(paste(IN_DIR,"ReactionNetwork_Rel.txt",sep=""))

In [3]:
%%R
rxn2nodeLabel.nls <- list()
nodeLabel2rxn.nls <- list()
for(i in 1:length(X)){
  rxn2nodeLabel.nls[[names(X)[i]]] <- i
  nodeLabel2rxn.nls[[i]] <- names(X)[i]
}

In [4]:
%%R
E <- E %>%
  dplyr::filter(V1 %in% names(rxn2nodeLabel.nls))
head(E)

             V1        V2            V3
1 R-MMU-9613507 Preceding R-MMU-9613545
2 R-MMU-9613545 Preceding R-MMU-9613562
3 R-MMU-9613562 Preceding R-MMU-9613670
4 R-MMU-9639715 Preceding R-MMU-9640117
5 R-MMU-9641089 Preceding R-MMU-9641096
6 R-MMU-9641096 Preceding R-MMU-9641111


In [5]:
%%R
E <- E %>%
  dplyr::filter(V3 %in% names(rxn2nodeLabel.nls))
head(E)

             V1        V2            V3
1 R-MMU-9613507 Preceding R-MMU-9613545
2 R-MMU-9613545 Preceding R-MMU-9613562
3 R-MMU-9613562 Preceding R-MMU-9613670
4 R-MMU-9639715 Preceding R-MMU-9640117
5 R-MMU-9641089 Preceding R-MMU-9641096
6 R-MMU-9641096 Preceding R-MMU-9641111


In [6]:
%%R
E <- E %>%
  dplyr::filter(V1 %in% names(rxn2nodeLabel.nls)) %>%
  dplyr::filter(V3 %in% names(rxn2nodeLabel.nls)) %>%
  dplyr::select(V1,V3)

In [7]:
%%R
write.table(E,
            file=paste(IN_DIR,"edgeLabels_time_course_0.03vs0.csv",sep=""),
            row.names = FALSE,
            col.names = FALSE)

In [8]:
%%R
node1 <- numeric()
node2 <- numeric()
for(i in 1:nrow(E)){
  node1 <- c(node1,rxn2nodeLabel.nls[[as.character(E$V1[i])]])
  node2 <- c(node2,rxn2nodeLabel.nls[[as.character(E$V3[i])]])
}

In [9]:
%%R
z <- unlist(rxn2nodeLabel.nls)
y <- unlist(nodeLabel2rxn.nls)
assertthat::are_equal(length(z),length(y))
head(z)

R-MMU-170666 R-MMU-170671 R-MMU-170674 R-MMU-170686 R-MMU-380073 R-MMU-392187 
           1            2            3            4            5            6 


In [10]:
%%R
write.table(z,
            file=paste(IN_DIR,"rxn2nodeLabel_nls_time_course_0.03vs0.csv",sep=""),
            row.names = TRUE,
            col.names = FALSE)
write.table(y,
            file=paste(IN_DIR,"nodeLabel2rxn_nls_time_course_0.03vs0.csv",sep=""),
            row.names = TRUE,
            col.names = FALSE)


In [11]:
%%R
E <- data.frame(node1 = node1,
                node2 = node2)

X <- as.data.frame(X)

Y <- as.data.frame(Y)

In [12]:
%%R
# write.table(E,
#           file=paste(IN_DIR,"edges.txt",sep=""),
#           row.names = FALSE,
#           col.names = FALSE)
write.table(X,
          file=paste(IN_DIR,"node_features0.03vs0_time_course.txt",sep=""),
          row.names = TRUE,
          col.names = TRUE)
write.table(Y,
          file=paste(IN_DIR,"graph_targets0.03vs0_time_course.txt",sep=""),
          row.names = FALSE,
          col.names = FALSE)

In [13]:
%%R
missing_pathway <- read.csv("/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/missing_columns0.03vs0_time_course.txt",sep="")
print(missing_pathway)

               x
1  R.MMU.1299359
2   R.MMU.429594
3  R.MMU.2465924
4  R.MMU.9656893
5  R.MMU.9648949
6  R.MMU.9648983
7  R.MMU.9649108
8   R.MMU.981832
9  R.MMU.9667756
10 R.MMU.5619420
11 R.MMU.9007882
12 R.MMU.9825608
13 R.MMU.9825624
14 R.MMU.6788295
15 R.MMU.8850794
16 R.MMU.1482679
17 R.MMU.1482776
18 R.MMU.1482907
19 R.MMU.8862771


In [14]:
%%R
all_values <- as.vector(as.matrix(X))
median_value <- median(all_values, na.rm = TRUE)
print(median_value)

[1] -0.06232156


In [15]:
%%R
for (col in missing_pathway$x) {
  X[[col]] <- median_value
}

In [16]:
%%R
write.table(X,
          file=paste(IN_DIR,"node_features0.03vs0_2_time_course.txt",sep=""),
          row.names = FALSE,
          col.names = FALSE)